In [1]:
import pandas as pd
import numpy as np
from numpy.random import default_rng
import seaborn as sns
import os
from os import path
sns.set()
rng = default_rng()

In [2]:
numberOfSamples = 10000
#filename = "../../results_all/micro_history_full_influxdb.csv"
#outFolderName = "../../results_all/micro_bootstrapping_influx"

filename = "../../results_all/micro_history_opti_influxdb.csv"
outFolderName = "../../results_all/micro_bootstrapping_influx_opti"

debug = False
slidingAvg=10
jumpThreshold = 4
trendThreshold = 2


# iterations of microbenchmarks
numberOfIterations = 5
CIsmall = 1

In [3]:
#! mkdir -p {outFolderName} # type: ignore

In [4]:
def resample(perfRuntimes1: pd.DataFrame,
            perfRuntimes2: pd.DataFrame,
            instanceRuns: np.ndarray,
            suiteRuns: np.ndarray,
            numberOfIterations: int,
            numberOfSamples: int) -> np.ndarray:
    """
        Resamples performances using hierarchical bootstrapping for building confindence intervals

        Builds a tensor of random indices of a form numberOfSamples * instanceRunsNumber * suiteRunsNumber * numberOfIterations.
        Then uses these indices to choose from a performance runtimes tensor with a form instanceRunsNumber * suiteRunsNumber * numberOfIterations.
        Finally, reshapes resulting tensor to a matrix of form numberOfSamples * ( instanceRunsNumber * suiteRunsNumber * numberOfIterations)
        and calculates performance differences between medians.

        Parameters
        ----------
        perfRuntimes1 : performance runtimes of the first version.
        perfRuntimes2 : performance runtimes of the second version.
        instanceRuns : array of instanceRun numbers
        suiteRuns : array of suiteRun numbers
        numberOfIterations : number of microbenchmark iterations

        Returns
        -------
        np.ndarray
            array of performance differences with a shape numberOfSample * 1.

        """
    instanceRunsNumber = instanceRuns.shape[0]
    suiteRunsNumber = suiteRuns.shape[0]

    allRuntimes1 = np.ndarray((instanceRunsNumber, suiteRunsNumber, numberOfIterations))
    allRuntimes2 = np.ndarray((instanceRunsNumber, suiteRunsNumber, numberOfIterations))

    #aggregate measurements from all instance and suite runs
    for instanceRun in instanceRuns:
        for suiteRun in suiteRuns:
            allRuntimes1[instanceRun - 1][suiteRun] = perfRuntimes1.loc[(perfRuntimes1['instanceRun'] == instanceRun)
                                                    & (perfRuntimes1['suiteRun'] == suiteRun),
                                                    'runtime'].to_numpy()
            allRuntimes2[instanceRun - 1][suiteRun] = perfRuntimes2.loc[(perfRuntimes2['instanceRun'] == instanceRun)
                                                    & (perfRuntimes2['suiteRun'] == suiteRun),
                                                    'runtime'].to_numpy()
    #Generate Random Arrays
    currentInstanceRun = rng.choice(instanceRuns, size=(instanceRunsNumber, numberOfSamples)) - 1
    currentSuiteRun = rng.choice(suiteRuns, size=(suiteRunsNumber, instanceRunsNumber, numberOfSamples))
    currentRuntimes1 = rng.integers(numberOfIterations, size=(numberOfIterations, suiteRunsNumber, instanceRunsNumber, numberOfSamples))
    currentRuntimes2 = rng.integers(numberOfIterations, size=(numberOfIterations, suiteRunsNumber, instanceRunsNumber, numberOfSamples))

    #Bulk selection
    tmp1 = allRuntimes1[currentInstanceRun, currentSuiteRun, currentRuntimes1]
    tmp1 = tmp1.reshape((numberOfSamples, suiteRunsNumber * instanceRunsNumber * numberOfIterations))

    tmp2 = allRuntimes2[currentInstanceRun, currentSuiteRun, currentRuntimes2]
    tmp2 = tmp2.reshape((numberOfSamples, suiteRunsNumber * instanceRunsNumber * numberOfIterations))
    # Get median for both lists
    med1 = np.median(tmp1, axis=1)
    med2 = np.median(tmp2, axis=1)

    return med2/med1

In [5]:
df_all = pd.read_csv(filename,sep=";")


# For each microbenchmark
for name in df_all.name.unique():
    print(f"Running analysis for benchmark {name}...")

    benchmarkMeasurements = df_all.loc[(df_all['name'].str.startswith(name, na=False))]
    benchmarkMeasurements.sort_values(by=["number"], inplace=True)
    instanceRuns = benchmarkMeasurements.instanceRun.unique()
    suiteRuns = benchmarkMeasurements.suiteRun.unique()


    if len(benchmarkMeasurements) > 50:
        printName = name.replace("/","-")
        printName = printName.replace("/","-")
        printName = printName.replace("\\","-")
        benchmarkFilename = os.path.join(outFolderName, printName + ".csv")
        if path.exists(benchmarkFilename):
            print("  Already analyzed, skip.")
        else:
            results = []
            # For each commit (number)
            lastValues = []

            for commitNumber in benchmarkMeasurements.number.unique():
                print(f"  Running analysis for commit {commitNumber}...")

                #Find median perf. change
                perfRuntimes1 = benchmarkMeasurements.loc[(benchmarkMeasurements['number'] == commitNumber)
                                                    & (benchmarkMeasurements['version'] == 1)]
                perfRuntimes2 = benchmarkMeasurements.loc[(benchmarkMeasurements['number'] == commitNumber)
                                                    & (benchmarkMeasurements['version'] == 2)]


                elements1 = perfRuntimes1['runtime'].shape[0]
                elements2 = perfRuntimes2['runtime'].shape[0]
                if (elements1 == 45) & (elements2 == 45):

                    perf1 = perfRuntimes1['runtime'].median()
                    perf2 = perfRuntimes2['runtime'].median()
                    # Compare both (e.g., 10ms in ver1 and 12ms in ver2 => 12/10 = 1.2 (>1 -> regression)
                    perfChange = ((perf2/perf1) - 1) * 100

                    print(f"    Found median performance change ({perfChange}).")

                    # Run Bootstrapping
                    # R stores the 10.000 median values
                    R = resample(perfRuntimes1=perfRuntimes1,
                            perfRuntimes2=perfRuntimes2,
                            instanceRuns=instanceRuns,
                            suiteRuns=suiteRuns,
                            numberOfIterations=numberOfIterations,
                            numberOfSamples=numberOfSamples)

                    print(f"    Bootstrapping done ({len(R)} elements in R).")

                    # Find conf. intervals
                    R.sort()

                    small = int((numberOfSamples * CIsmall) / 100 / 2)
                    if small == 0:
                        small  = 1

                    minSmall = R[small-1]
                    minSmall = (minSmall - 1) * 100
                    maxSmall = R[numberOfSamples-small-1]
                    maxSmall = (maxSmall - 1) * 100
                    print(f"    Found conf. intervals: ([{minSmall}, {maxSmall}]).")

                    lastValues.append({
                        "commit": commitNumber,
                        "min": minSmall,
                        "med": perfChange,
                        "max": maxSmall,
                    })

                    # Jump detection
                    jump = ""
                    if (len(lastValues) > 1):
                        currVal = lastValues[len(lastValues)-1]['med']
                        prevVal = lastValues[len(lastValues)-2]['med']
                        diff = currVal - prevVal
                        #print(f"diff is {diff}. {prevVal} -> {currVal}")
                        if (diff > jumpThreshold):
                            jump = "potential up"
                        if ((-1 * diff) > jumpThreshold):
                            jump = "potential down"
                        if (jump != ""):
                            # check CIs
                            currMin = lastValues[len(lastValues)-1]['min']
                            currMax = lastValues[len(lastValues)-1]['max']
                            prevMin = lastValues[len(lastValues)-2]['min']
                            prevMax = lastValues[len(lastValues)-2]['max']
                            if (currMin > prevMax):
                                jump = "definite up"
                            if (currMax < prevMin):
                                jump = "definite down"

                    if (jump != ""):
                        print(f"    Found {jump} jump at commit {commitNumber}.")

                    # Trend detection
                    trend = ""
                    #Clear values if there is a definite jump
                    if (jump.startswith("definite")):
                        lastValues = lastValues[-1:]

                    if (len(lastValues) > 2):
                        currVal = lastValues[len(lastValues)-1]['med']
                        sumOfPrevVals = 0
                        for val in lastValues[:-1]:
                            sumOfPrevVals += val['med']
                        diff = currVal - (sumOfPrevVals / (len(lastValues)-1))
                        #print(f"diff is {diff}. {sumOfPrevVals / slidingAvg} -> {currVal}")
                        if (diff > trendThreshold):
                            trend = "potential up"
                        if ((-1 * diff) > trendThreshold):
                            trend = "potential down"
                        if (trend != ""):
                            currMin = lastValues[len(lastValues)-1]['min']
                            currMax = lastValues[len(lastValues)-1]['max']
                            for val in lastValues[:-1]:
                                prevMin = val['min']
                                prevMax = val['max']
                                if (currMin > prevMax):
                                    trend = "definite up"
                                if (currMax < prevMin):
                                    trend = "definite down"


                    if (trend != ""):
                        print(f"    Found {trend} trend at commit {commitNumber}.")

                    # Remove first element
                    if (len(lastValues) > slidingAvg):
                        lastValues.pop(0)

                    #Store values
                    results.append({
                                    "name" : name,
                                    "number" : int(commitNumber),
                                    "min" : minSmall,
                                    "med" : perfChange,
                                    "max" : maxSmall,
                                    "jump": jump,
                                    "trend": trend
                                    })
                else:
                    print(f"  Skip {name};{commitNumber}, only ({elements1}, {elements2}) elements.")

            #Store results for this benchmark to file
            # Convert to data frame
            df_results = pd.DataFrame(results)
            if len(results) > 0:
                df_results.sort_values(by=["name","number"], inplace=True)
                df_results.describe()

                #Create folder
                #Save file
                df_results.to_csv(benchmarkFilename, sep=";")

    else:
        print(f"  Skip {name}, only {len(benchmarkMeasurements)} measurments.")

Running analysis for benchmark BenchmarkCreateIterator/tsi1_shards_1-2...
  Already analyzed, skip.
Running analysis for benchmark BenchmarkCreateIterator/tsi1_shards_2-2...
  Already analyzed, skip.
Running analysis for benchmark BenchmarkCreateIterator/tsi1_shards_3-2...
  Already analyzed, skip.
Running analysis for benchmark BenchmarkCreateIterator/tsi1_shards_4-2...
  Already analyzed, skip.
Running analysis for benchmark BenchmarkCreateIterator/tsi1_shards_5-2...
  Already analyzed, skip.
Running analysis for benchmark BenchmarkDecodeFloatArrayBlock/1000-2...
  Already analyzed, skip.
Running analysis for benchmark BenchmarkDecodeFloatArrayBlock/5-2...
  Already analyzed, skip.
Running analysis for benchmark BenchmarkDecodeFloatArrayBlock/55-2...
  Already analyzed, skip.
Running analysis for benchmark BenchmarkDecodeFloatArrayBlock/555-2...
  Already analyzed, skip.
Running analysis for benchmark BenchmarkIndex_IndexFile_TagValueSeriesIDIterator/78888_series_TagValueSeriesIDIter

<ipython-input-5-87164366bd4d>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  benchmarkMeasurements.sort_values(by=["number"], inplace=True)


In [6]:
print("Done.")


Done.
